In [8]:
# Getting input from a file on local computer

p = []
g = []

with open("input_file.txt") as f: 
        count=0
        for lines in f:
            if(count == 0):
                p = lines.strip(',')
            if(count == 1):
                g = lines.strip(',')
            if(count == 2):
                temp = int(lines.strip(',')[slice(3)])
            if(count == 3):
                h_1 = lines.strip(',')[slice(9)]
            if(count == 4):
                h_2 = lines.strip(',')[slice(9)]
            if(count == 5):
                neighbour_function = lines.strip(',')
            if(count == 6):
                probability_function = lines.strip(',')
            if(count == 7):
                cool_f = lines.strip(',')[slice(9)]
            count = count + 1

# getting list of state elements as integer values from string 
puzzle = []
goal = []
for i in range(1,26,3):
    puzzle.append(int(p[i]))
    goal.append(int(g[i]))

hf = {h_1:1,h_2:2}        # assigning heuristic functions a value for referencing, by creating a dictionary

In [9]:
print("Start:",puzzle,"\nGoal:",goal)
print("heuristic functions:",hf)

Start: [1, 2, 3, 5, 6, 0, 7, 8, 4] 
Goal: [1, 2, 3, 5, 8, 6, 0, 7, 4]
heuristic functions: {'Manhattan': 1, 'Misplaced': 2}


In [10]:
class Puzzle:
    
    goal_state=None
    hn = None
    hn_value=None
    
    def __init__(self,state,parent,action,path_cost):     # initiater of class object
        
        self.state = state
        self.parent = parent
        self.action = action
        if parent:
            self.path_cost = parent.path_cost + path_cost
        else:
            self.path_cost = path_cost
        
        if self.hn == 1:
            self.manhattan_hn()
        elif self.hn == 2:
            self.misplaced_hn()

    def __str__(self):                        #representing class object as customized string
        
        return str(self.state[0:3])+'\n'+str(self.state[3:6])+'\n'+str(self.state[6:9])
    
    
    def manhattan_hn(self):                   #manhattan distance heuristic calculation
        
        self.hn_value=0
        for num in range(1,9):
            distance = abs(self.state.index(num) - self.goal_state.index(num))
            i = int(distance/3)
            j = int(distance%3)
            self.hn_value = self.hn_value +i +j
        
        
    def misplaced_hn(self):                    #no. of misplaced tiles heuristic calculation
        
        sum = 0
        for i in range(9):
            if(self.state[i] == 0): continue
            if(self.state[i] != self.goal_state[i]): sum+=1
        self.hn_value = sum

        
    def goal_test(self):                       #check goal_state reached or not
        
        if self.state == self.goal_state:
            return True
        return False

    
    @staticmethod
    def find_legal_actions(i,j):               #finds legal actions from a particular tile in a puzzle
        
        legal_action = ['U', 'D', 'L', 'R']
        if i == 0:                             # up is illegal
            legal_action.remove('U')
        elif i == 2:                           # down is illegal
            legal_action.remove('D')
        if j == 0:                             # left is illegal
            legal_action.remove('L')
        elif j == 2:                           # right is illegal
            legal_action.remove('R')
        return legal_action
    
    
    def generate_child(self):                   #generate childs of state
        
        children=[]
        x = self.state.index(0)                 #getting index of element 0 i.e blank tile in linear array
        i = int(x / 3)
        j = int(x % 3)
        legal_actions = self.find_legal_actions(i,j)

        for action in legal_actions:            #for every legal action creating a child with new state
            new_state = self.state.copy()
            if action == 'U':
                new_state[x], new_state[x-3] = new_state[x-3], new_state[x]
            elif action == 'D':
                new_state[x], new_state[x+3] = new_state[x+3], new_state[x]
            elif action == 'L':
                new_state[x], new_state[x-1] = new_state[x-1], new_state[x]
            elif action == 'R':
                new_state[x], new_state[x+1] = new_state[x+1], new_state[x]
            children.append(Puzzle(new_state,self,action,1))
        return children

    
    def find_solution(self):                    #find optimal path if goal_state reached
        
        solution = []
        solution.append(self.action)            #storing the action taken to come to this current state
        path = self
        while path.parent != None:              #backtracking to the start_state
            path = path.parent         
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution

In [11]:

def Hill_climb(initial_state,goal_state,hn):
    
    Puzzle.goal_state = goal_state               #setting the value of class variable goal_state
    Puzzle.hn = hn                               #setting the value of class variable hn(heuristic function preference)
    
    explored=[]                                  #list of states explored i.e. present in closed list
    start_node=Puzzle(initial_state,None,None,0) #initializing start_state object
    goal_node=Puzzle(goal_state,None,None,0)
    
    open_list = []                               #creating open list
    open_list.append((start_node.hn_value,start_node))#inserting tuple(hn_value,initial_state)

    t0 = time()
    count=0
    
    while len(open_list)>0:
        node=open_list[0][1]                       #accessing state element from list
        open_list.pop()                            #removing state element from list
        explored.append(node.state)                #inserting state element into closed/explored state
        
        if node.goal_test():                     #checking the state for goal_state
            print("Puzzle solved successfully by Hill climbing\n")
            print("Start State\n",start_node,"\n")
            print("Goal State\n",node,"\n")
            print('Toatal states generated :',count)
            print('Total no. of states explored by Hill climbing : ',len(explored)-1)
            opt_path = node.find_solution()
            print('Total number of states to optimal path: ',len(opt_path))
            print('Optimal path:',opt_path)
            print('Optimal path cost: ',node.path_cost)
            return
        
        elif time()-t0>20:
            print("Start State\n",start_node,"\n")
            print("Goal State\n",goal_node,"\n")
            print('Toatal states generated :',count)
            print("Puzzle not solvable within time limit")
            print('Total no. of states explored by Hill climbing before termination : ',len(explored)-1)
            return
        
        children=node.generate_child()          #generating childs of state if its not a goal state
        count += len(children)
        prev_value = node.hn_value
        best_child = None
        l_c = 0
        
        for child in children:                  # for every child if not explored, insert it in the open list
            curr_value = child.hn_value
            if curr_value < prev_value:
                min_value = curr_value
                best_child = child
                l_c+=1
                
        
        if(l_c == 0):
            print("Start State\n",start_node,"\n")
            print("Goal State\n",goal_node,"\n")            
            print('Toatal states generated :',count)
            print("Puzzle not solvable because local optima reached")
            print("Local optima state\n",node,"\n")
            print('Total no. of states explored by Hill climbing before termination : ',len(explored)-1)
            return
        open_list.append((best_child.hn_value,best_child))
    return

In [12]:
from time import time

start_state= puzzle;
goal_state= goal;

print(h_1,"Heuristic function used\n")

t0 = time()

Hill_climb(start_state,goal_state,hf[h_1])
t1 = time() - t0

print('Time taken: ', t1)

print()
print(h_2,"Heuristic function used\n")

Hill_climb(start_state,goal_state,hf[h_2])
t2 = time() - t0 -t1
print('Time taken: ', t2)


Manhattan Heuristic function used

Puzzle solved successfully by Hill climbing

Start State
 [1, 2, 3]
[5, 6, 0]
[7, 8, 4] 

Goal State
 [1, 2, 3]
[5, 8, 6]
[0, 7, 4] 

Toatal states generated : 10
Total no. of states explored by Hill climbing :  3
Total number of states to optimal path:  3
Optimal path: ['L', 'D', 'L']
Optimal path cost:  3
Time taken:  0.002000570297241211

Misplaced Heuristic function used

Puzzle solved successfully by Hill climbing

Start State
 [1, 2, 3]
[5, 6, 0]
[7, 8, 4] 

Goal State
 [1, 2, 3]
[5, 8, 6]
[0, 7, 4] 

Toatal states generated : 10
Total no. of states explored by Hill climbing :  3
Total number of states to optimal path:  3
Optimal path: ['L', 'D', 'L']
Optimal path cost:  3
Time taken:  0.0019941329956054688
